<a href="https://colab.research.google.com/github/MILLERMARRU/paddle_ocr/blob/main/padle_ocr_stream_lit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# 1. Instala las dependencias
!pip install -q paddleocr
!pip install -q paddlepaddle -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install -q streamlit pyngrok python-docx opencv-python



In [27]:
!ngrok config add-authtoken 2rPOA88C1NvlhE7pXulxY6VlUMA_4LMGUuiywutna8pigaZ4A

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [87]:
from pathlib import Path

app_code = """

# app.py
from paddleocr import PaddleOCR
from PIL import Image
import streamlit as st
import numpy as np
from docx import Document
from io import BytesIO
from pathlib import Path

@st.cache_resource
def load_ocr():
    return PaddleOCR(
        use_doc_orientation_classify=False,
        use_doc_unwarping=False,
        use_textline_orientation=False
    )

def generar_docx(texto):
    doc = Document()
    doc.add_paragraph(texto)
    buffer = BytesIO()
    doc.save(buffer)
    buffer.seek(0)
    return buffer

ocr = load_ocr()

st.set_page_config(page_title="OCR con PaddleOCR", layout="wide")
st.title("📄 OCR con PaddleOCR")

if "descargado" not in st.session_state:
    st.session_state.descargado = False

uploaded_file = st.file_uploader("📤 Sube una imagen", type=["png", "jpg", "jpeg"])

col1, col2 = st.columns([2, 3])  # 40% imagen, 60% texto

if uploaded_file:
    try:
        image = Image.open(uploaded_file).convert("RGB")
        image.save("input.jpg")

        with st.spinner("🔍 Procesando imagen..."):
            result = ocr.predict(input="input.jpg")[0]

        textos_detectados = result['rec_texts']
        result.save_to_img("resultado_ocr.jpg")

        with col1:
            st.image("resultado_ocr.jpg", caption="🖼️ Imagen con cuadros", use_container_width=True)

        with col2:
            st.markdown("📝 **Texto reconocido (editable)**")
            texto_completo = "\\n".join(textos_detectados)
            texto_editado = st.text_area("Resultado OCR:", texto_completo, height=400)

            # ✅ Solo un botón de descarga
            buffer = generar_docx(texto_editado)
            # Botón de descarga
            downloaded = st.download_button(
                label="📥 Descargar como .docx",
                data=buffer,
                file_name="resultado_ocr.docx",
                mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
            )

            # Reiniciar app después de descarga
            if downloaded:
                st.success("✅ Documento descargado. Reinicia para procesar otra imagen.")
                st.session_state.descargado = True
                st.experimental_rerun()


    except Exception as e:
        st.error(f"❌ Error procesando la imagen.\\n\\n{e}")
else:
    st.info("👆 Por favor, sube una imagen para empezar.")

# Si se pidió reiniciar
if st.session_state.descargado:
    st.session_state.clear()
    st.experimental_rerun()



"""

Path("app.py").write_text(app_code)


2455

In [86]:
import os
import time
import threading
from pyngrok import ngrok

# Paso 1: Mata túneles anteriores y Streamlit previos
ngrok.kill()
!pkill streamlit

# Paso 2: Función para ejecutar Streamlit en un hilo separado
def run_streamlit():
    os.system('streamlit run app.py --server.port=8504')

# Paso 3: Lanzar Streamlit en segundo plano
thread = threading.Thread(target=run_streamlit)
thread.start()

# Paso 4: Espera para que Streamlit se inicie
time.sleep(5)

# Paso 5: Abre el túnel ngrok
public_url = ngrok.connect(8504)
print(f"🚀 Tu app está corriendo en: {public_url}")



🚀 Tu app está corriendo en: NgrokTunnel: "https://3868e452d075.ngrok-free.app" -> "http://localhost:8504"
